In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# per optimizzazione
from pyomo.environ import *

In [2]:
## Set declaration
# set of airports
N=[i for i in range(10)]
# set of flight legs
I=[(n1, n2) for n1 in N for n2 in N if n1!=n2] # does n1=origin and n2=destination?
# set of aircraft types
A=['A', 'B', 'C']

In [3]:
w={'A':100, 'B':200, 'C':50} 

# from first toy simulations: highest capacity is assigned highest number of flights 
# i.e. use more aircrafts of higher capacity

In [4]:
d={}
for i in I: 
    d[i]=100+np.random.rand()*1000 #why did we use 100+? --< intercept : point to start from , can be random 
    # make sure A-B = B-A

In [5]:
p={}
for i in I:
    for a in A: 
        p[i,a]=5+np.random.rand()*100 #and why 5 here? 

In [6]:
r={}
for i in I:
    for a in A: 
        if np.random.rand()<=0.5:
            r[i,a]=0
        else: 
            r[i,a]=1

In [7]:
# Creation of a Concrete Model
m = ConcreteModel()
m.I=Set(initialize=I)
m.A=Set(initialize=A)
m.w=Param(m.A, initialize=w)
m.d=Param(m.I, initialize=d)
m.p=Param(m.I, m.A, initialize=p)
m.r=Param(m.I, m.A, initialize=r)

In [8]:
m.f=Var(m.I, m.A, domain=NonNegativeIntegers)
m.z=Var(m.A, domain=NonNegativeIntegers)

In [9]:
def minimize_emission(m):
    return sum(m.f[i,a]*m.p[i,a] for i in m.I for a in m.A)
m.objective = Objective(rule=minimize_emission, sense=minimize)

In [10]:
def utilization_rule(m, a): 
    return sum(m.f[i, a] for i in m.I)<=10*m.z[a] # 10 = k -> max utilization of aircraft type a

m.utilization = Constraint(m.A, rule=utilization_rule)

In [11]:
def demand_rule(m, i1, i2): 
    return sum(m.f[i1, i2, a]*m.w[a] for a in m.A)>=m.d[i1,i2]

m.demand = Constraint(m.I, rule=demand_rule)

In [12]:
''' NOT WORKING
def operation_rule(m, i1, i2, a):
    return sum(m.f[i1, i2, a] for a in m.A for i in m.I<=m.r[i1,i2, a]*m.f[i1, i2, a])

m.operation = Constraint(m.I, m.A, rule=operation_rule)
'''

' NOT WORKING\ndef operation_rule(m, i1, i2, a):\n    return sum(m.f[i1, i2, a] for a in m.A for i in m.I<=m.r[i1,i2, a]*m.f[i1, i2, a])\n\nm.operation = Constraint(m.I, m.A, rule=operation_rule)\n'

In [13]:
#SolverFactory?

In [14]:
%%time

#Solve using neos-cplex
#opt = SolverFactory('cplex')
opt = SolverFactory('cbc', executable=r'/usr/local/bin/cbc')
opt.options['mipgap'] = 0.20
#solver_manager = SolverManagerFactory('neos')       
results = opt.solve(m, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Aug 12 2020 

command line - /usr/local/bin/cbc -mipgap 0.2 -printingOptions all -import /var/folders/wt/wvx51pxn2213wbhxthm8s1380000gn/T/tmpc5z7s8hn.pyomo.lp -stat=1 -solve -solu /var/folders/wt/wvx51pxn2213wbhxthm8s1380000gn/T/tmpc5z7s8hn.pyomo.soln (default strategy 1)
No match for mipgap - ? for list of commands
No match for 0.2 - ? for list of commands
Option for printingOptions changed from normal to all
Presolve 90 (-4) rows, 270 (-4) columns and 270 (-274) elements
Statistics for presolved model
Original problem has 273 integers (0 of which binary)
Presolved problem has 270 integers (0 of which binary)
==== 0 zero objective 270 different
==== absolute objective values 270 different
==== for integers 0 zero objective 270 different
==== for integers absolute objective values 270 different
===== end objective counts


Problem has 90 rows, 270 columns (270 with objective) and 270 elements
There are 270 singletons with obj

In [15]:
for a in m.A:
    print(a)
    print(m.z[a].value)

A
11.0
B
21.0
C
11.0


In [16]:
for i in m.I:
    for a in m.A:
        print('%s-%s: %s' % (i, a, m.f[i,a].value))

(0, 1)-A: 0.0
(0, 1)-B: 2.0
(0, 1)-C: 0.0
(0, 2)-A: 0.0
(0, 2)-B: 3.0
(0, 2)-C: 0.0
(0, 3)-A: 11.0
(0, 3)-B: 0.0
(0, 3)-C: 0.0
(0, 4)-A: 5.0
(0, 4)-B: 0.0
(0, 4)-C: 0.0
(0, 5)-A: 0.0
(0, 5)-B: 3.0
(0, 5)-C: 0.0
(0, 6)-A: 1.0
(0, 6)-B: 5.0
(0, 6)-C: 0.0
(0, 7)-A: 0.0
(0, 7)-B: 4.0
(0, 7)-C: 0.0
(0, 8)-A: 2.0
(0, 8)-B: 0.0
(0, 8)-C: 0.0
(0, 9)-A: 0.0
(0, 9)-B: 1.0
(0, 9)-C: 0.0
(1, 0)-A: 0.0
(1, 0)-B: 5.0
(1, 0)-C: 0.0
(1, 2)-A: 1.0
(1, 2)-B: 5.0
(1, 2)-C: 0.0
(1, 3)-A: 0.0
(1, 3)-B: 5.0
(1, 3)-C: 0.0
(1, 4)-A: 1.0
(1, 4)-B: 5.0
(1, 4)-C: 0.0
(1, 5)-A: 3.0
(1, 5)-B: 0.0
(1, 5)-C: 0.0
(1, 6)-A: 11.0
(1, 6)-B: 0.0
(1, 6)-C: 0.0
(1, 7)-A: 0.0
(1, 7)-B: 3.0
(1, 7)-C: 0.0
(1, 8)-A: 0.0
(1, 8)-B: 4.0
(1, 8)-C: 1.0
(1, 9)-A: 0.0
(1, 9)-B: 4.0
(1, 9)-C: 0.0
(2, 0)-A: 0.0
(2, 0)-B: 3.0
(2, 0)-C: 0.0
(2, 1)-A: 0.0
(2, 1)-B: 4.0
(2, 1)-C: 0.0
(2, 3)-A: 0.0
(2, 3)-B: 2.0
(2, 3)-C: 0.0
(2, 4)-A: 6.0
(2, 4)-B: 0.0
(2, 4)-C: 0.0
(2, 5)-A: 0.0
(2, 5)-B: 0.0
(2, 5)-C: 14.0
(2, 6)-A: 0.0
(2, 6)-B: 5.0
(2,

In [ ]:
#help --solvers